# BERT 영화 리뷰 감성 분석 Fine-tuning

이 노트북은 기존 Movie_Bert.ipynb와 같은 모델과 데이터를 사용하되, fine-tuning을 추가로 수행합니다.

## 기존 방식과의 차이점:
- 동일한 모델: BertTokenizer, BertModel ("monologg/koelectra-base-v3-discriminator")
- 동일한 데이터: NSMC 데이터셋
- **추가 사항**: End-to-end fine-tuning을 통한 성능 향상

In [ ]:
# 필요한 라이브러리 import (ELECTRA 모델 사용으로 수정)
import numpy as np
import pandas as pd
from transformers import (ElectraTokenizer, ElectraModel, ElectraForSequenceClassification, 
                         get_linear_schedule_with_warmup)
from torch.optim import AdamW  # ✅ PyTorch에서 가져오기

import warnings
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

warnings.filterwarnings('ignore')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

사용 디바이스: cpu


In [ ]:
# ELECTRA 모델 사용으로 수정 (BertTokenizer -> ElectraTokenizer, BertModel -> ElectraModel)
from transformers import ElectraTokenizer, ElectraModel

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
 
model = ElectraModel.from_pretrained(
    "monologg/koelectra-base-v3-discriminator", 
    output_hidden_states=True,
    use_safetensors=True
)

print("ELECTRA 모델 로드 완료!")
print(f"토크나이저 어휘 크기: {tokenizer.vocab_size}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.

기존 모델 로드 완료!
토크나이저 어휘 크기: 35000


In [ ]:
train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
train_df = pd.read_csv(train_url, sep="\t")

test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
test_df = pd.read_csv(test_url, sep="\t")

print("원본 데이터:")
print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(train_df.head())

# NaN 값 제거 (중요: 인코딩 전에 먼저 제거)
print(f"\nNaN 값 제거 전 - Train: {len(train_df)}, Test: {len(test_df)}")
train_df = train_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)
print(f"NaN 값 제거 후 - Train: {len(train_df)}, Test: {len(test_df)}")

# 샘플 추출
train_df = train_df.sample(6000, random_state=42).reset_index(drop=True)
test_df = test_df.sample(2000, random_state=42).reset_index(drop=True)

print(f"\n최종 샘플 크기 - Train: {len(train_df)}, Test: {len(test_df)}")
print("샘플 데이터:")
print(train_df.head())

원본 데이터:
Train: 150000, Test: 50000
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

NaN 값 제거 전 - Train: 150000, Test: 50000
NaN 값 제거 후 - Train: 149995, Test: 49997

최종 샘플 크기 - Train: 6000, Test: 2000
샘플 데이터:
        id                    document  label
0  7865795                      원본이 최고      1
1  5417631            스릴감과 훈훈함이 있는 영화.      1
2  8357466      굉장히 저평가되는 영화중 하나라고 생각함      1
3  8252946  정말영화같은이야기 영화여서 영화같은이야기가 좋다      1
4  7800452                 계기도없는데 이상하다      0


In [ ]:
train_encodings = []
for i, text in enumerate(tqdm(train_df['document'], desc="Train 데이터 인코딩 중")):
    # NaN 체크 제거 (이미 전처리에서 제거됨)
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding='max_length', max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    train_encodings.append(outputs.pooler_output.squeeze().numpy())

test_encodings = []
for i, text in enumerate(tqdm(test_df['document'], desc="Test 데이터 인코딩 중")):
    # NaN 체크 제거 (이미 전처리에서 제거됨)
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding='max_length', max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    test_encodings.append(outputs.pooler_output.squeeze().numpy())

# 크기 확인
print(f"\n인코딩 결과:")
print(f"Train 인코딩: {len(train_encodings)}개 (DF: {len(train_df)}개)")
print(f"Test 인코딩: {len(test_encodings)}개 (DF: {len(test_df)}개)")
print(f"임베딩 차원: {train_encodings[0].shape if train_encodings else 'N/A'}")

Test 데이터 인코딩 중: 100%|██████████| 2000/2000 [10:30<00:00,  3.17it/s]


In [24]:
# 로지스틱 회귀로 분류
print(len(train_encodings))
logistic = LogisticRegression(max_iter=1000)
logistic.fit(train_encodings, train_df['label'])
preds = logistic.predict(test_encodings)
baseline_accuracy = accuracy_score(test_df['label'], preds)
print(f"\n기존 방식 정확도: {baseline_accuracy:.4f}")

5999


ValueError: Found input variables with inconsistent numbers of samples: [5999, 6000]

In [ ]:
# Fine-tuning을 위한 데이터셋 클래스 정의
class NSMCDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]
        
        # 토크나이징 (기존 방식과 동일)
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("NSMCDataset 클래스 정의 완료!")

In [ ]:
# Fine-tuning용 ELECTRA 모델 설정
print("=== Fine-tuning용 ELECTRA 모델 설정 ===")

# ElectraForSequenceClassification 모델 로드
fine_tune_model = ElectraForSequenceClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=2,
    use_safetensors=True
)

fine_tune_model.to(device)

# 데이터 준비 (더 작은 샘플 사용으로 빠른 실험)
train_subset = train_df.sample(n=2000, random_state=42)
test_subset = test_df.sample(n=800, random_state=42)

# 데이터로더 생성
train_dataset = NSMCDataset(train_subset['document'], train_subset['label'], tokenizer)
test_dataset = NSMCDataset(test_subset['document'], test_subset['label'], tokenizer)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Fine-tuning 데이터:")
print(f"- 훈련: {len(train_subset)}개")
print(f"- 테스트: {len(test_subset)}개")
print(f"- 배치 크기: {batch_size}")
print(f"- 모델 유형: ELECTRA")

In [ ]:
# Fine-tuning 설정
epochs = 2
learning_rate = 2e-5

optimizer = AdamW(fine_tune_model.parameters(), lr=learning_rate, eps=1e-8)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"Fine-tuning 설정:")
print(f"- 에폭: {epochs}")
print(f"- 학습률: {learning_rate}")
print(f"- 총 스텝: {total_steps}")
print(f"- 모델 파라미터 수: {sum(p.numel() for p in fine_tune_model.parameters()):,}")

In [ ]:
# 훈련 및 평가 함수 정의
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    progress_bar = tqdm(data_loader, desc="Training")
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )
        
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        predictions = torch.argmax(logits, dim=-1)
        correct_predictions += torch.sum(predictions == labels).item()
        total_predictions += labels.size(0)
        
        current_accuracy = correct_predictions / total_predictions
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{current_accuracy:.4f}'
        })
    
    return total_loss / len(data_loader), correct_predictions / total_predictions

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )
            
            loss = outputs.loss
            logits = outputs.logits
            
            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=-1)
            correct_predictions += torch.sum(predictions == labels).item()
            total_predictions += labels.size(0)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct_predictions / total_predictions
    avg_loss = total_loss / len(data_loader)
    
    return avg_loss, accuracy, all_predictions, all_labels

print("훈련 및 평가 함수 정의 완료!")

In [ ]:
# Fine-tuning 실행
print("=" * 60)
print("BERT 모델 Fine-tuning 시작")
print("=" * 60)

# 훈련 결과 저장
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 30)
    
    # 훈련
    train_loss, train_acc = train_epoch(fine_tune_model, train_loader, optimizer, scheduler, device)
    
    # 평가
    val_loss, val_acc, _, _ = evaluate_model(fine_tune_model, test_loader, device)
    
    # 결과 저장
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

end_time = time.time()
training_time = end_time - start_time
print(f"\n총 훈련 시간: {training_time:.2f}초 ({training_time/60:.2f}분)")

print("\nFine-tuning 완료!")

In [ ]:
# 최종 평가 및 비교
_, final_accuracy, predictions, true_labels = evaluate_model(fine_tune_model, test_loader, device)

print("=" * 60)
print("최종 성능 비교")
print("=" * 60)
print(f"기존 방식 (고정 임베딩 + 로지스틱 회귀): {baseline_accuracy:.4f}")
print(f"Fine-tuning 방식: {final_accuracy:.4f}")
print(f"성능 향상: {final_accuracy - baseline_accuracy:.4f} ({(final_accuracy - baseline_accuracy)/baseline_accuracy*100:+.1f}%)")

# 분류 보고서
print("\n=== Fine-tuned 모델 분류 보고서 ===")
target_names = ['부정', '긍정']
print(classification_report(true_labels, predictions, target_names=target_names))

In [ ]:
# 결과 시각화
plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('BERT Fine-tuning 결과 분석', fontsize=16, fontweight='bold')

# 1. 훈련 손실 변화
axes[0, 0].plot(range(1, epochs + 1), train_losses, 'b-o', label='Train Loss', linewidth=2)
axes[0, 0].plot(range(1, epochs + 1), val_losses, 'r-o', label='Validation Loss', linewidth=2)
axes[0, 0].set_title('Loss 변화', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 정확도 변화
axes[0, 1].plot(range(1, epochs + 1), train_accuracies, 'b-o', label='Train Accuracy', linewidth=2)
axes[0, 1].plot(range(1, epochs + 1), val_accuracies, 'r-o', label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('Accuracy 변화', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Confusion Matrix
cm = confusion_matrix(true_labels, predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['부정', '긍정'], yticklabels=['부정', '긍정'], ax=axes[1, 0])
axes[1, 0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('예측')
axes[1, 0].set_ylabel('실제')

# 4. 성능 비교
methods = ['기존 방식\n(고정 임베딩)', 'Fine-tuning\n방식']
accuracies = [baseline_accuracy, final_accuracy]
colors = ['lightcoral', 'lightblue']

bars = axes[1, 1].bar(methods, accuracies, color=colors, alpha=0.7)
axes[1, 1].set_title('성능 비교', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('정확도')
axes[1, 1].set_ylim(0, 1)

# 막대 위에 값 표시
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 요약 통계
print("\n=== 훈련 요약 ===")
print(f"최종 훈련 정확도: {train_accuracies[-1]:.4f}")
print(f"최종 검증 정확도: {val_accuracies[-1]:.4f}")
print(f"최종 훈련 손실: {train_losses[-1]:.4f}")
print(f"최종 검증 손실: {val_losses[-1]:.4f}")
print(f"총 훈련 시간: {training_time:.2f}초")
print(f"에폭당 평균 시간: {training_time/epochs:.2f}초")

In [ ]:
# Fine-tuned 모델로 새로운 텍스트 예측
def predict_sentiment_finetuned(text, model, tokenizer, device):
    """
    Fine-tuned 모델을 사용한 감성 분석 함수
    """
    model.eval()
    
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        prediction = torch.argmax(logits, dim=-1).item()
        confidence = probabilities[0][prediction].item()
    
    sentiment = "긍정" if prediction == 1 else "부정"
    
    return {
        'text': text,
        'sentiment': sentiment,
        'confidence': confidence,
        'probabilities': {
            '부정': probabilities[0][0].item(),
            '긍정': probabilities[0][1].item()
        }
    }

# 테스트 예제
test_examples = [
    "이 영화는 정말 재미있었어요! 강력 추천합니다.",
    "너무 지루하고 시간 낭비였어요.",
    "배우들의 연기가 훌륭했습니다.",
    "스토리가 엉성하고 재미없었어요.",
    "감동적이고 의미 있는 영화였습니다."
]

print("=" * 60)
print("Fine-tuned 모델 감성 분석 예제")
print("=" * 60)

for i, text in enumerate(test_examples, 1):
    result = predict_sentiment_finetuned(text, fine_tune_model, tokenizer, device)
    print(f"\n{i}. 텍스트: {result['text']}")
    print(f"   예측: {result['sentiment']} (신뢰도: {result['confidence']:.4f})")
    print(f"   확률 - 부정: {result['probabilities']['부정']:.4f}, 긍정: {result['probabilities']['긍정']:.4f}")

print("\n" + "=" * 60)

## 결론

### 기존 방식과 Fine-tuning 비교:

**기존 방식 (Movie_Bert.ipynb):**
- 고정된 BERT 임베딩 추출
- 로지스틱 회귀로 분류
- 빠른 실행 시간
- 제한적인 성능

**Fine-tuning 방식:**
- End-to-end 학습
- 전체 모델 파라미터 업데이트
- 더 긴 훈련 시간
- 향상된 성능

### 주요 개선사항:
1. **성능 향상**: Fine-tuning을 통한 정확도 개선
2. **도메인 적응**: 영화 리뷰 데이터에 특화된 모델
3. **신뢰도**: 더 높은 예측 신뢰도

### 사용 권장사항:
- **빠른 프로토타이핑**: 기존 방식 사용
- **높은 성능 요구**: Fine-tuning 방식 사용
- **실제 서비스**: Fine-tuning 후 모델 배포